# 0. `TildeRPairNormalizer`

In [1]:
import numpy as np
from typing import List, Dict

from matersdk.io.pwmat.output.movement import Movement
from matersdk.io.publicLayer.structure import DStructure
from matersdk.io.publicLayer.neigh import StructureNeighborsDescriptor
from matersdk.feature.deepmd.se_pair import DpseTildeRPairDescriptor
from matersdk.feature.deepmd.preprocess import TildeRPairNormalizer

# 1. Custom parameters

In [2]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/matersdk/demo/feature/movement/LiSi.config"
movement_path = "/data/home/liuhanyu/hyliu/code/mlff/test/demo2/PWdata/data1/MOVEMENT"
movement = Movement(movement_path=movement_path)

# 2. Initialize `StructureNeighbors`
1. Note: We just use the first 10 frame to calculate `davg` and `dstd`

In [3]:
def concat_tildeRs(
            structure_indices:List[int],
            rcut:float,
            rcut_smooth:float,
            center_atomic_number:int,
            nbr_atomic_numbers:List[int],
            scaling_matrix:List[int],
            max_num_nbrs_dict:Dict[int, int]
            ):
    ### Step 1. 得到 DStructure 的列表
    structures_lst = [movement.get_frame_structure(idx_frame=tmp_idx) for tmp_idx in structure_indices]

    ### Step 2. 
    tildeRs_lst = []
    for tmp_structure in structures_lst:
        tmp_struct_nbr = StructureNeighborsDescriptor.create(
                        'v1',
                        structure=tmp_structure,
                        rcut=rcut,
                        scaling_matrix=scaling_matrix,
                        reformat_mark=True,
                        coords_are_cartesian=True)
        
        tmp_tildeRs_lst = []
        for tmp_nbr_an in nbr_atomic_numbers:
            tmp_tildeR_pair = DpseTildeRPairDescriptor.create(
                        'v1',
                        structure_neighbors=tmp_struct_nbr,
                        center_atomic_number=center_atomic_number,
                        nbr_atomic_number=tmp_nbr_an,
                        rcut=rcut,
                        rcut_smooth=rcut_smooth).get_tildeR(
                                        max_num_nbrs=max_num_nbrs_dict[tmp_nbr_an])
            tmp_tildeRs_lst.append(tmp_tildeR_pair)
        # shape = (48, 180, 4)
        tmp_tildeR_tot = np.concatenate(tmp_tildeRs_lst, axis=1)
        tildeRs_lst.append(tmp_tildeR_tot)
    # shape = (48, 1800, 4)
    tildeR_tot = np.concatenate(tildeRs_lst, axis=1)
    return tildeR_tot

In [7]:
structure_indices = [*range(10)]
rcut = 3.0
rcut_smooth = 3.0
center_atomic_number = 3    # Li
nbr_atomic_numbers = [3, 14]      # Li, Si
scaling_matrix = [3, 3, 3]
max_num_nbrs = {3: 100, 14: 100}

reformat_mark = True
coords_are_cartesian = True

In [8]:
tildeRs_array = concat_tildeRs(
                structure_indices=structure_indices,
                rcut=rcut,
                rcut_smooth=rcut_smooth,
                center_atomic_number=center_atomic_number,
                nbr_atomic_numbers=nbr_atomic_numbers,
                scaling_matrix=scaling_matrix,
                max_num_nbrs_dict=max_num_nbrs)
tildeRs_array.shape

(48, 2000, 4)

# 3. Initialize `DpseTildeRPairNormalizer`

In [9]:
normalizer = TildeRPairNormalizer(tildeRs_array=tildeRs_array)
print("Step 1.1. davg of Environment matrix : ", end='\n\t')
print(normalizer.davg)
print("Step 1.2. dstd of Environment matrix : ", end='\n\t')
print(normalizer.dstd)

Step 1.1. davg of Environment matrix : 
	[[0.01554078 0.         0.         0.        ]]
Step 1.2. dstd of Environment matrix : 
	[[0.07365668 0.04346195 0.04346195 0.04346195]]


# 4. Normalize a new `tildeR` of new `DStructure`

In [86]:
new_structure = movement.get_frame_structure(idx_frame=100)
struct_nbr = StructureNeighborsDescriptor.create(
                'v1',
                structure=new_structure,
                rcut=rcut,
                scaling_matrix=scaling_matrix,
                reformat_mark=reformat_mark,
                coords_are_cartesian=coords_are_cartesian)

new_tildeRs_array = DpseTildeRPairDescriptor.create(
                'v1',
                structure_neighbors=struct_nbr,
                center_atomic_number=3,
                nbr_atomic_number=14,
                rcut=rcut,
                rcut_smooth=rcut_smooth).get_tildeR(max_num_nbrs=100)

In [87]:
print("Step 2. Using a new environment matrix, after normalize...")
print("Step 2.1. The max value of environment is : ", end="\n\t")
print(np.max(normalizer.normalize(tildeRs_array=new_tildeRs_array)))
print("Step 2.2. The min value of environment is : ", end="\n\t")
print(np.min(normalizer.normalize(tildeRs_array=new_tildeRs_array)))

Step 2. Using a new environment matrix, after normalize...
Step 2.1. The max value of environment is : 
	5.252378253804327
Step 2.2. The min value of environment is : 
	-5.221954388104391
